In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [2]:
IMG_SIZE = 224
BATCH_SIZE = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=25,
    zoom_range=0.2,
    horizontal_flip=True
)

train_data = datagen.flow_from_directory(
    "dataset_mendeley",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)

val_data = datagen.flow_from_directory(
    "dataset_mendeley",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)

Found 19764 images belonging to 2 classes.
Found 4941 images belonging to 2 classes.


In [3]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False

x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(train_data.num_classes, activation="softmax")(x)

model = tf.keras.models.Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [4]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

Epoch 1/10
618/618 ━━━━━━━━━━━━━━━━━━━━ 354s 567ms/step - accuracy: 0.9040 - loss: 0.2427 - val_accuracy: 0.9170 - val_loss: 0.2168
Epoch 2/10
618/618 ━━━━━━━━━━━━━━━━━━━━ 444s 719ms/step - accuracy: 0.9290 - loss: 0.1845 - val_accuracy: 0.9118 - val_loss: 0.2271
Epoch 3/10
618/618 ━━━━━━━━━━━━━━━━━━━━ 329s 533ms/step - accuracy: 0.9382 - loss: 0.1644 - val_accuracy: 0.8960 - val_loss: 0.2574
Epoch 4/10
618/618 ━━━━━━━━━━━━━━━━━━━━ 816s 1s/step - accuracy: 0.9392 - loss: 0.1571 - val_accuracy: 0.9192 - val_loss: 0.2123
Epoch 5/10
618/618 ━━━━━━━━━━━━━━━━━━━━ 339s 548ms/step - accuracy: 0.9431 - loss: 0.1500 - val_accuracy: 0.9114 - val_loss: 0.2198
Epoch 6/10
618/618 ━━━━━━━━━━━━━━━━━━━━ 743s 1s/step - accuracy: 0.9445 - loss: 0.1418 - val_accuracy: 0.9027 - val_loss: 0.2425
Epoch 7/10
618/618 ━━━━━━━━━━━━━━━━━━━━ 356s 577ms/step - accuracy: 0.9471 - loss: 0.1371 - val_accuracy: 0.8867 - val_loss: 0.2750
Epoch 8/10
618/618 ━━━━━━━━━━━━━━━━━━━━ 319s 515ms/step - accuracy: 0.9490 - loss:

In [5]:
base_model.trainable = True

for layer in base_model.layers[:-30]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history_fine = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5
)

Epoch 1/5
618/618 ━━━━━━━━━━━━━━━━━━━━ 372s 592ms/step - accuracy: 0.9315 - loss: 0.1790 - val_accuracy: 0.9294 - val_loss: 0.2089
Epoch 2/5
618/618 ━━━━━━━━━━━━━━━━━━━━ 410s 664ms/step - accuracy: 0.9432 - loss: 0.1523 - val_accuracy: 0.9197 - val_loss: 0.2356
Epoch 3/5
618/618 ━━━━━━━━━━━━━━━━━━━━ 840s 1s/step - accuracy: 0.9490 - loss: 0.1309 - val_accuracy: 0.9235 - val_loss: 0.2237
Epoch 4/5
618/618 ━━━━━━━━━━━━━━━━━━━━ 375s 606ms/step - accuracy: 0.9542 - loss: 0.1223 - val_accuracy: 0.9304 - val_loss: 0.2213
Epoch 5/5
618/618 ━━━━━━━━━━━━━━━━━━━━ 370s 599ms/step - accuracy: 0.9569 - loss: 0.1127 - val_accuracy: 0.9231 - val_loss: 0.2320


In [6]:
model.save("organic_model.keras")